In [1]:
# feature_{0...129} - represents real stock data. 
# Each row represnts trading opportunity, for which you will be predicting an action value: 1 to make the trade and 0 to pass on it. 
# Weight and resp - represents a return on the trade. 
# date - day of the trade
# ts_id = time ordering
# 

# resp_{1,2,3,4} - returns over different time horizons. 
#

# https://www.kaggle.com/fernandocanteruccio/jane-street-reinforce-agent-exploration -- Solving the problem using Reinforcement learning 

In [2]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

feature_data = pd.read_csv('./features.csv')
feature_data.head()

,feature,tag_0,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6,tag_7,tag_8,...,tag_19,tag_20,tag_21,tag_22,tag_23,tag_24,tag_25,tag_26,tag_27,tag_28
0,feature_0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,feature_1,False,False,False,False,False,False,True,True,False,...,False,False,False,False,False,False,False,False,False,False
2,feature_2,False,False,False,False,False,False,True,True,False,...,False,False,False,False,False,False,False,False,False,False
3,feature_3,False,False,False,False,False,False,True,False,True,...,False,False,False,False,False,False,False,False,False,False
4,feature_4,False,False,False,False,False,False,True,False,True,...,False,False,False,False,False,False,False,False,False,False


In [9]:
import pandas as pd

data_train_original = pd.read_csv('./train.csv')
data_train_sliced = data_train_original.head(10000)
data_train_sliced

,date,weight,resp_1,resp_2,resp_3,resp_4,resp,feature_0,feature_1,feature_2,...,feature_121,feature_122,feature_123,feature_124,feature_125,feature_126,feature_127,feature_128,feature_129,ts_id
0,0,0.000000,0.009916,0.014079,0.008773,0.001390,0.006270,1,-1.872746,-2.191242,...,NaN,1.168391,8.313583,1.782433,14.018213,2.653056,12.600292,2.301488,11.445807,0
1,0,16.673515,-0.002828,-0.003226,-0.007319,-0.011114,-0.009792,-1,-1.349537,-1.704709,...,NaN,-1.178850,1.777472,-0.915458,2.831612,-1.417010,2.297459,-1.304614,1.898684,1
2,0,0.000000,0.025134,0.027607,0.033406,0.034380,0.023970,-1,0.812780,-0.256156,...,NaN,6.115747,9.667908,5.542871,11.671595,7.281757,10.060014,6.638248,9.427299,2
3,0,0.000000,-0.004730,-0.003273,-0.000461,-0.000476,-0.003200,-1,1.174378,0.344640,...,NaN,2.838853,0.499251,3.033732,1.513488,4.397532,1.266037,3.856384,1.013469,3
4,0,0.138531,0.001252,0.002165,-0.001215,-0.006219,-0.002604,1,-3.172026,-3.093182,...,NaN,0.344850,4.101145,0.614252,6.623456,0.800129,5.233243,0.362636,3.926633,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1,0.420165,0.001847,0.016171,0.039813,0.061120,0.051909,-1,-0.803361,-1.687988,...,-2.735500,-0.603269,-1.588024,-0.614948,-2.783072,-0.882524,-1.890361,-0.789504,-1.662447,9995
9996,1,2.852259,-0.000712,-0.004864,-0.009795,-0.018601,-0.019537,1,-0.702446,-1.161989,...,-0.861067,-0.883083,-0.165111,-0.779975,-0.422061,-0.968161,0.167733,-0.920668,-0.010404,9996
9997,1,1.389255,0.002383,0.004190,0.002356,0.006833,0.009608,1,0.130875,-0.608811,...,0.276647,-0.001041,0.269809,-0.036464,0.415885,0.056424,0.422491,-0.036431,0.176114,9997
9998,1,0.085839,-0.003532,-0.037491,-0.053815,-0.052844,-0.065972,1,-3.172026,-3.093182,...,0.027756,0.940655,-0.102939,0.714865,-0.302707,1.022478,-0.315486,0.806448,-0.441524,9998


In [10]:
data_train_sliced.to_csv('train_sliced.csv',index=False)

In [11]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

data_train = pd.read_csv('./train_sliced.csv')

In [12]:
# EDA on the dataset

data_train.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Columns: 138 entries, date to ts_id
dtypes: float64(135), int64(3)
memory usage: 10.5 MB


In [2]:
# Do feature engineering
# 1. Check the feature correlations. 
# 2. 
data_train['action'] = ((data_train['weight'].values * data_train['resp'].values) > 0).astype('int')


In [3]:
X = data_train.loc[:,data_train.columns.str.contains('feature')]
Y = data_train.loc[:,'action']

In [5]:
#split data into train and test dataset

seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=test_size, random_state=seed)

In [6]:
# fit model to training data
model = XGBClassifier()
model.fit(X_train, y_train)

print(model)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)


In [7]:
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

In [8]:
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 63.48%
